In [56]:
import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import Agent
import time

class Blackboard(Agent):

    def on_init(self):
        self.ka_to_execute = (None, 0)
        self.trigger_event_num = 0
        self.trigger_events = {}
        self.pub_trigger_alias = 'trigger'
        self.pub_trigger_addr = self.bind('PUB', alias=self.pub_trigger_alias)

    def connect_pub_trigger(self, message):
        agent_name, response_addr, response_alias = message
        self.connect(response_addr, alias=response_alias, handler=self.response_trigger_handler)
        return (self.pub_trigger_alias, self.pub_trigger_addr)

    def send_pub_trigger(self):
        self.send(self.pub_trigger_alias, 'sending trigger')

    def response_trigger_handler(self, message):
        self.log_info('Writing Trigger Response')
        agent, trig_val = message
    
    def run_problem(self):
        
        for i in range(2):
            #self.log('Starting Timer')
            self.send_pub_trigger()
            time.sleep(1)

        
class KaBase(Agent):
    
    def on_init(self):
        self.trigger_val = 0
        self.trigger_response_alias = None
        self.trigger_response_addr = None
    
    def add_blackboard(self, blackboard):
        self.bb = blackboard
    
    def connect_pub_trigger(self):
        if self.bb:
            self.trigger_response_alias = 'trigger_response_{}'.format(self.name)
            self.trigger_response_addr = self.bind('PUSH', alias=self.trigger_response_alias)
            self.pub_trigger_alias, self.pub_trigger_addr = self.bb.connect_pub_trigger((self.name, self.trigger_response_addr, self.trigger_response_alias))
            self.connect(self.pub_trigger_addr, alias=self.pub_trigger_alias, handler=self.pub_trigger_handler)
    
    def pub_trigger_handler(self, message):
        self.log_info('Sent trigger handler')
        self.send(self.trigger_response_alias, (self.name, self.trigger_val))

In [57]:
ns = run_nameserver()
bb = run_agent(name='blackboard', base=Blackboard)
ka_rp = run_agent(name='ka_rp', base=KaBase)

ka_rp.add_blackboard(bb)
ka_rp.connect_pub_trigger()

for i in range(2):
    bb.send_pub_trigger()
    time.sleep(1)

ns.shutdown()

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:17733 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:17733
INFO [2020-02-26 16:47:29.409920] (ka_rp): Sent trigger handler
INFO [2020-02-26 16:47:29.411943] (blackboard): Writing Trigger Response
INFO [2020-02-26 16:47:30.413244] (ka_rp): Sent trigger handler
INFO [2020-02-26 16:47:30.415158] (blackboard): Writing Trigger Response
NS shut down.


In [58]:
ns = run_nameserver()
bb = run_agent(name='blackboard', base=Blackboard)
ka_rp = run_agent(name='ka_rp', base=KaBase)

ka_rp.add_blackboard(bb)
ka_rp.connect_pub_trigger()

bb.run_problem()

ns.shutdown()

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:18727 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:18727
INFO [2020-02-26 16:47:34.983973] (ka_rp): Sent trigger handler
INFO [2020-02-26 16:47:35.985655] (ka_rp): Sent trigger handler
INFO [2020-02-26 16:47:36.987399] (blackboard): Writing Trigger Response
INFO [2020-02-26 16:47:36.990119] (blackboard): Writing Trigger Response
NS shut down.
